# Stats on wind and storm events


In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import copy as cp
import pandas as pd
import countryinfo
import geopandas as gpd
from timeit import default_timer as timer # try to measure time
from climada.engine import Impact
from os import mkdir, remove, rmdir


In [2]:
##function definition
from functions import *
from climada_functions import *
from constants import * 

In [78]:
## select data
#select variable (cmip6 naming)
selvar = 'sfcWindmax'
pathinvar = pathcmip6+'sfcWindmax/'

#preprocessing 
gst_fact = 1.67
qt = 0.98
cut=2E6
min_lat=30
max_lat=75
min_lon=-30
max_lon=30

##climada constants
haz_type = 'WS'
haz_id = 1

## naming
#name base (meteo) variable
metvar = [cmip6vars[selvar]]
spaceres = ["br_rg"] #base resolution regridded
timeres = ["day"]
domain = ["EU"]
season = ["winE"]
scen = ["allscens"]
sep = "_"
lst_bn = metvar+spaceres+timeres+domain+season+scen
basenamemet = sep.join(lst_bn)

#preproc field
processings = ["qt"+str(qt)[-2:]+"pst","cutarea"+format(cut,'.0E').replace("+0",''),"gst1-67"]
basenamemet_proc = make_fn(processings,basenamemet)


In [79]:
#select climate models, used scenarios (historical, ssp585,...), impact functions and calibration type
modlist = modlist_allscen
scen_used= ["historical", "ssp585"]
proc_funcs = {"mask_qt":mask_qt, "scale_qt":scale_qt}
pastname = 'historical'
timeres='day'
stack = True
nmems = 3
qmin = 0.025
qmax = 0.975

In [80]:
#initiate df to save results
proc = proc_funcs.keys() #preprocessing on the hazard data
stat = ["daily freq","max int", "med int","min int"]
itercol = [scen_used,proc,stat]
col_idx = pd.MultiIndex.from_product(itercol,names=["scenario","processing","stat"])
iterrows = [modlist,range(nmems)]
row_idx= pd.MultiIndex.from_product(iterrows,names=["model","member"])


In [81]:
## iterate over hazard data

#initiate df
if stack:
    res_df = pd.DataFrame(columns=col_idx,index=modlist)
    nmems=1
else:
    res_df = pd.DataFrame(columns=col_idx,index=row_idx)

for modid, modname in enumerate(modlist):
    
    #read netcdf
    fn = make_fn([modname],basenamemet,filetype=".nc")
    ncdf = xr.open_dataset(pathinvar+fn)
    ndays = 3*len(ncdf.day)
    #iterate over the scenarios
    for scen in scen_used:
        #select simulation
        ncdf_sel = ncdf[[pastname,scen]]
        #apply gust factor
        gust_ds = gst_fact*ncdf_sel
        
        #iterate over the members
        for imem in range(nmems):
            print('iterating over member: '+str(imem))
            
            #preprocess fields
            if stack:
                mem_da = gust_ds
            else: 
                mem_da = gust_ds.sel(member=imem)
            for funcname,func in proc_funcs.items():
                gust_pp = func(mem_da,qt,cutarea=cut,timeres=timeres,pastname=pastname,futname=scen,stack=stack)
                gust_pp_sel = gust_pp[scen]
                
                #prepare hazards centroids
                haz = set_centroids(gust_pp_sel,stack=stack,timeres=timeres,plot=False)
                
                nb_events = len(haz.event_id)
                max_ev = np.nanquantile(haz.intensity.data,qmax)
                min_ev = np.nanquantile(haz.intensity.data,qmin)
                mean_ev = np.nanquantile(haz.intensity.data,0.5)

                if stack:
                    res_df.loc[modname,(scen,funcname,"daily freq")] = nb_events/ndays
                    res_df.loc[modname,(scen,funcname,"max int")] = max_ev
                    res_df.loc[modname,(scen,funcname,"min int")] = min_ev
                    res_df.loc[modname,(scen,funcname,"med int")] = mean_ev
                else:
                    res_df.loc[(modname,imem),(scen,funcname,"daily freq")] = nb_events/ndays
                    res_df.loc[(modname,imem),(scen,funcname,"max int")] = max_ev
                    res_df.loc[(modname,imem),(scen,funcname,"min int")] = min_ev
                    res_df.loc[(modname,imem),(scen,funcname,"med int")] = mean_ev


                del haz
                    

iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0
iterating over member: 0


In [82]:
res_df.loc["mean"] = res_df.mean()

In [83]:
res_df

scenario         historical                                              \
processing          mask_qt                                    scale_qt   
stat             daily freq    max int    med int    min int daily freq   
CanESM5            0.158088   49.65069  38.481238  16.140256   0.158088   
CNRM-CM6-1         0.160629  48.436812  36.047832  14.784572   0.160629   
CNRM-ESM2-1        0.164288  48.273145  36.355594   14.70731   0.164288   
EC-Earth3-Veg      0.137212  49.757037  36.446498   14.84206   0.137212   
EC-Earth3-Veg-LR    0.15386  49.018292  35.829463  14.473163    0.15386   
IPSL-CM6A-LR       0.166667  46.954827  35.059546  14.809453   0.166667   
MIROC-ES2L         0.178192  55.759603  37.471127  15.174992   0.178192   
UKESM1-0-LL        0.158889  44.928504  33.339276   11.81417   0.158889   
MRI-ESM2-0          0.15075  60.617139  42.995439   11.79142    0.15075   
FGOALS-g3          0.189229  43.165362   30.99611  15.541787   0.189229   
ACCESS-ESM1-5      0.174351  45.662072  32.661018  11.267234   0.174351   
MIROC6              0.15569  51.435308  32.531722   9.202667    0.15569   
MPI-ESM1-2-LR      0.176546    49.4489  36.181038  15.408073   0.176546   
KACE-1-0-G         0.156852  44.513319  33.220032  12.096787   0.156852   
mean               0.162946  49.115786  35.543995  13.718139   0.162946   

scenario                                           ssp585             \
processing                                        mask_qt              
stat               max int   med int   min int daily freq    max int   
CanESM5           0.394305  0.084177  0.003582   0.292361  47.414289   
CNRM-CM6-1        0.338677  0.082803  0.003544   0.399378  46.154169   
CNRM-ESM2-1       0.342311  0.084507  0.003648   0.364618  46.756478   
EC-Earth3-Veg     0.394562  0.095093  0.003895    0.33187  48.043455   
EC-Earth3-Veg-LR  0.381857  0.095414  0.003971    0.32199   46.81248   
IPSL-CM6A-LR      0.407516  0.090732  0.003787   0.399561  47.254299   
MIROC-ES2L        0.413166  0.103168   0.00411   0.311014  55.573882   
UKESM1-0-LL       0.402683    0.0858  0.003487   0.301667  43.188287   
MRI-ESM2-0        0.431046  0.098341  0.004022   0.378339  56.867843   
FGOALS-g3         0.319596  0.078749  0.003244   0.399341  42.245372   
ACCESS-ESM1-5     0.353052  0.083327  0.003378   0.400659  43.634538   
MIROC6            0.405247  0.099033  0.004032   0.335346  50.337338   
MPI-ESM1-2-LR     0.349723  0.089899  0.003759   0.391145  48.183957   
KACE-1-0-G        0.403214  0.087427  0.003594   0.335741  42.797316   
mean              0.381211  0.089891  0.003718   0.354502  47.518836   

scenario                                                               \
processing                               scale_qt                       
stat                med int    min int daily freq   max int   med int   
CanESM5           36.231292   16.61863   0.292361  0.396209  0.074161   
CNRM-CM6-1        31.114457   13.81681   0.399378  0.330009  0.077501   
CNRM-ESM2-1       33.155651  13.989902   0.364618  0.325598  0.074292   
EC-Earth3-Veg     34.553863  12.825976    0.33187  0.397883    0.0852   
EC-Earth3-Veg-LR  32.902717  12.063777    0.32199  0.373245  0.081443   
IPSL-CM6A-LR      35.004579  14.375944   0.399561  0.377981  0.082057   
MIROC-ES2L        39.631349  13.168547   0.311014  0.391762  0.091601   
UKESM1-0-LL       32.236942  10.065721   0.301667  0.393502  0.072404   
MRI-ESM2-0        36.127449  10.931948   0.378339  0.423875  0.087242   
FGOALS-g3         29.984988  15.295807   0.399341  0.296905  0.069141   
ACCESS-ESM1-5     28.411571  10.921478   0.400659  0.339726   0.07432   
MIROC6            29.561493    9.02616   0.335346   0.39596  0.086919   
MPI-ESM1-2-LR     34.988356  14.857389   0.391145  0.335487  0.078243   
KACE-1-0-G        33.321196  11.712524   0.335741  0.342627  0.072513   
mean              33.373279  12.833615   0.354502  0.365769  0.079074   

scenario                    
processi

In [77]:
res_df

scenario         historical                                              \
processing          mask_qt                                    scale_qt   
stat             daily freq    max int    med int    min int daily freq   
CanESM5            0.426269  48.488438  36.744762   16.13907   0.426269   
CNRM-CM6-1         0.444932   47.48963  34.232706  14.370821   0.444932   
CNRM-ESM2-1         0.44603  47.561321  34.477936  14.151384    0.44603   
EC-Earth3-Veg      0.438163  48.777787  35.098059   14.04218   0.438163   
EC-Earth3-Veg-LR   0.434504  47.891186  34.260138  13.463988   0.434504   
IPSL-CM6A-LR        0.44109  46.202184  34.202791  14.800252    0.44109   
MIROC-ES2L          0.44603  54.604983  36.724007  15.263899    0.44603   
UKESM1-0-LL        0.430185  44.104576  31.266849  10.556145   0.430185   
MRI-ESM2-0         0.431028  59.610197  41.165253  11.534506   0.431028   
FGOALS-g3          0.461989  42.554026  29.980232  15.354918   0.461989   
ACCESS-ESM1-5       0.47768  45.121801   30.23117  11.068477    0.47768   
MIROC6             0.439078  50.562346  31.501698   9.064903   0.439078   
MPI-ESM1-2-LR      0.442737  48.559749  35.024279  15.225493   0.442737   
KACE-1-0-G         0.432963  43.795647  31.474669  10.766968   0.432963   
mean               0.442334  48.237419  34.027468  13.271643   0.442334   

scenario                                           ssp585             \
processing                                        mask_qt              
stat               max int   med int   min int daily freq    max int   
CanESM5           0.393553  0.077156  0.003026    0.61275  46.858925   
CNRM-CM6-1        0.316735  0.073865  0.003047   0.711855  45.536405   
CNRM-ESM2-1       0.318356  0.074154  0.003048   0.681486  46.111479   
EC-Earth3-Veg     0.363664  0.083103  0.003344   0.690084  47.316032   
EC-Earth3-Veg-LR  0.354731  0.083438  0.003354   0.663922  46.092413   
IPSL-CM6A-LR      0.374242  0.079733  0.003229   0.697219  46.638285   
MIROC-ES2L        0.383955  0.090813   0.00363   0.624954  54.398183   
UKESM1-0-LL        0.37647  0.075606  0.002963    0.63537  42.510797   
MRI-ESM2-0        0.397062  0.085872  0.003389   0.708196  55.961877   
FGOALS-g3         0.296793  0.069273   0.00272    0.70251   41.86886   
ACCESS-ESM1-5     0.329741  0.073041  0.002879   0.741859  43.060593   
MIROC6            0.373983  0.086085  0.003417   0.665752  49.878565   
MPI-ESM1-2-LR     0.329097  0.078664  0.003195   0.694841   47.56172   
KACE-1-0-G        0.371827  0.076588  0.003091   0.685185  42.262406   
mean              0.355729  0.079099  0.003167   0.679713  46.861182   

scenario                                                               \
processing                               scale_qt                       
stat                med int    min int daily freq   max int   med int   
CanESM5            34.34309  16.368719    0.61275  0.392196  0.070158   
CNRM-CM6-1        29.897817  13.507361   0.711855  0.313417  0.069184   
CNRM-ESM2-1       31.902441  13.622767   0.681486  0.310258  0.067211   
EC-Earth3-Veg     32.547434  12.092871   0.690084  0.376057  0.077583   
EC-Earth3-Veg-LR  31.495585  11.466261   0.663922  0.351258  0.073659   
IPSL-CM6A-LR      34.451834  14.188948   0.697219  0.360649  0.074526   
MIROC-ES2L         39.39094  12.638369   0.624954  0.363245  0.080778   
UKESM1-0-LL       30.451311   9.090033    0.63537  0.392968  0.067302   
MRI-ESM2-0        34.578355  10.760243   0.708196  0.399735  0.078298   
FGOALS-g3         29.483431  15.145635    0.70251  0.280183  0.062538   
ACCESS-ESM1-5     27.182707  10.773818   0.741859  0.322048  0.067345   
MIROC6            29.327884   8.918466   0.665752  0.370444  0.078968   
MPI-ESM1-2-LR     34.129841  14.694851   0.694841  0.318876  0.071446   
KACE-1-0-G        32.209753  10.868412   0.685185   0.32828   0.06667   
mean              32.242316   12.43834   0.679713  0.348544  0.071833   

scenario                    
processi